# Important Packages to Install Before Starting the Code

In [ ]:
# !pip install pandas numpy matplotlib scikit-learn keras tensorflow gensim seaborn

# Import All the Important Packages

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import re
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras import layers , activations , models , preprocessing, utils
from tensorflow.keras.layers import GRU , MultiHeadAttention, TimeDistributed ,Input, Dense, Input, Embedding, LSTM, Dense, Concatenate, AdditiveAttention , Bidirectional, concatenate , Dropout , Activation, dot, concatenate
from tensorflow.keras.layers import Attention
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Attention, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop

# Translation Files

## Novel Book Translation

- **Language**: English
- **File Name**: urdu.txt
- **Content**: The text of the novel in English.

## Bible Translation

- **Language**: Urdu
- **File Name**: bible..xlsx
- **Content**: The translation of the Bible in Urdu.


## You Can Load Files Individually

In [56]:
# lines=pd.read_table('urdu.txt', names=['eng', 'urdu'],index_col=False,encoding = 'utf-16')

In [57]:
# # Read the Excel file
# lines = pd.read_excel('cleaned_data.xlsx')

## You Can also train by merging file

In [65]:
import pandas as pd

# Read the Excel file
excel_data = pd.read_excel('bible.xlsx')

# Read the text file
text_data = pd.read_table('urdu.txt', names=['eng', 'urdu'], index_col=False, encoding='utf-16')

# Merge the data
merged_data = pd.concat([excel_data, text_data])

# Optionally, reset index
merged_data.reset_index(drop=True, inplace=True)

# Save the merged data as Excel file
merged_data.to_excel('english_to_urdu_dataset.xlsx', index=False)

print("Merged data saved as english_to_urdu_dataset.xlsx")


Merged data saved as english_to_urdu_dataset.xlsx


In [66]:
# Read the Excel file
lines = pd.read_excel('english_to_urdu_dataset.xlsx')

# Data Pre-Processing

In [67]:
# Convert numerical values to strings
lines['urdu'] = lines['urdu'].astype(str)

# Lowercase all characters
lines['eng'] = lines['eng'].apply(lambda x: x.lower())
lines['urdu'] = lines['urdu'].apply(lambda x: x.lower())

# Remove quotes
lines['eng'] = lines['eng'].apply(lambda x: re.sub("'", '', x))
lines['urdu'] = lines['urdu'].apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation)  # Set of all special characters

# Remove all the special characters
lines['eng'] = lines['eng'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['urdu'] = lines['urdu'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['eng'] = lines['eng'].apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines['eng'] = lines['eng'].apply(lambda x: x.strip())
lines['urdu'] = lines['urdu'].apply(lambda x: x.strip())
lines['eng'] = lines['eng'].apply(lambda x: re.sub(" +", " ", x))
lines['urdu'] = lines['urdu'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['urdu'] = lines['urdu'].apply(lambda x: 'START_ ' + x + ' _END')

lines.sample(10)

,eng,urdu
561,and when the disciples heard it they fell on t...,START_ شاگرد یہ سن کر منہ کے بل گرے اور بہت سے...
4877,do we then make void the law through faith god...,START_ پس کیا ہم شریعت کو ایمان سے باطل کرتے ہ...
6608,if thou put the brethren in remembrance of the...,START_ اگر تو بھائیوں کو یہ باتیں یاد دلائے گا...
1116,and again he entered into capernaum after some...,START_ کئی دن بعد جب وہ کفرنحوم میں پھر داخل ہ...
4027,and said behold i see the heavens opened and t...,START_ کہا کہ دیکھو ۔ میں آسمان کو کھلا اور اب...
398,and when he sowed some seeds fell by the way s...,START_ اور بوتے وقت کچھ دانے راہ کے کنارے گرے ...
5519,have all the gifts of healing do all speak wit...,START_ کیا سب کو شفا دینے کی قوّت عنایت ہوئی ک...
2853,saying the son of man must be delivered into t...,START_ ضرور ہے کہ ابن آدم گنہگاروں کے ہاتھ میں...
6920,for this melchisedec king of salem priest of t...,START_ اور یہ ملک صدق سالم کا بادشاہ ۔ خدا تعا...
8500,thats not the right answer,START_ یہ صحیح جواب نہیں ہے۔ _END


In [68]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Urdu 
all_urdu_words=set()
for urdu in lines.urdu:
    for word in urdu.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)

In [69]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print('Max Source Length:',max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.urdu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max Target Lenght:',max_length_tar)

Max Source Length: 68
Max Target Lenght: 86


In [70]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_urdu_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_urdu_words)+1
num_encoder_tokens, num_decoder_tokens

num_decoder_tokens += 1 # For zero padding
num_decoder_tokens


input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)
lines.head(10)

,eng,urdu
136,and if ye salute your brethren only what do ye...,START_ اور اگر تم فقط اپنے بھائیوں ہی کو سلام ...
1884,as it is written in the book of the words of e...,START_ جیسا یسعیاہ نبی کے کلام کی کتاب میں لکھ...
420,but when the blade was sprung up and brought f...,START_ پس جب پتّیا نکلیں اور بالیں آئیں تو وہ ...
7829,and every island fled away and the mountains w...,START_ اور ہر ایک ٹاپو اپنی جگہ سے ٹل گیا اور ...
411,for verily i say unto you that many prophets a...,START_ کیونکہ میں تم سے سچ کہتا ہوں کہ بہت سے ...
8300,is it a recent picture,START_ یہ ایک حالیہ تصویر ہے؟ _END
3917,but peter said ananias why hath satan filled t...,START_ مگر پطرس نے کہا اے حننیاہ ۔ کیوں شیطان ...
2863,and he said unto them what manner of communica...,START_ اس نے ان سے کہا یہ کیا باتیں ہیں جو تم ...
6978,whereupon neither the first testament was dedi...,START_ اسی لئے پہلا عہد بھی بغیر خون کے نہیں ب...
5783,wherefore though i wrote unto you i did it not...,START_ پس اگرچہ میں نے تم کو لکھا تھا مگر نہ ا...


# Split Data into Training and Testing for Model Evaluation

In [71]:

# Train - Test Split
X, y = lines.eng, lines.urdu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape


#Save the train and test dataframes for reproducing the results later, as they are shuffled.

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [72]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Set the Model Complexity According to Your Adjustment

In [76]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 500
latent_dim = 128
print(train_samples//batch_size)
print(val_samples//batch_size)

128
14


# Prepared Model of LSTM with Attention Mechanism

In [77]:
# Define the encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Attention layer
attn_layer = Attention(name='attention_layer')
attn_out = attn_layer([decoder_outputs, encoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# Dense layer
decoder_dense = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model with a RMSprop optimizer and a suitable learning rate
rmsprop_optimizer = RMSprop(lr=0.001)
model.compile(optimizer=rmsprop_optimizer, loss='categorical_crossentropy', metrics=['acc'])

In [78]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/500
128/128 [==============================] - 268s 2s/step - loss: 6.5106 - acc: 0.0620 - val_loss: 5.9141 - val_acc: 0.0771
Epoch 2/500
128/128 [==============================] - 268s 2s/step - loss: 5.9049 - acc: 0.0783 - val_loss: 5.8795 - val_acc: 0.0830
Epoch 3/500
128/128 [==============================] - 289s 2s/step - loss: 5.8580 - acc: 0.0907 - val_loss: 5.8303 - val_acc: 0.1029
Epoch 4/500
128/128 [==============================] - 289s 2s/step - loss: 5.7796 - acc: 0.1108 - val_loss: 5.7503 - val_acc: 0.1112
Epoch 5/500
128/128 [==============================] - 280s 2s/step - loss: 5.7022 - acc: 0.1165 - val_loss: 5.6736 - val_acc: 0.1199
Epoch 6/500
128/128 [==============================] - 266s 2s/step - loss: 5.6086 - acc: 0.1245 - val_loss: 5.5788 - val_acc: 0.1280
Epoch 7/500
128/128 [==============================] - 266s 2s/step - loss: 5.5132 - acc: 0.1317 - val_loss: 5.4667 - val_acc: 0.1357
Epoch 8/500
128/128 [==============================] - 266s 2s

# Always Remember to Save the Model so You Don't Need to Train Again

In [32]:
# Save model
model.save('model_with_attention.h5')

In [ ]:
model = load_model('model_with_attention.h5')

# This is the Inference Model in Encoder and Decoder

In [33]:
# Inference models
# For the encoder model, the outputs are the encoder outputs as well to be used in attention
encoder_model = Model(encoder_inputs, [encoder_outputs] + encoder_states)

# Decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_length_src, latent_dim))

# Get the embeddings and apply the decoder LSTM
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# Attention inference
attention_result_inf = attention_layer([decoder_outputs2, decoder_hidden_state_input])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attention_result_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [34]:
# Custom loop to decode a batch of sequences
def decode_sequence_batch(input_seqs):
    # Encode the input as state vectors
    encoder_outputs, states_value_h, states_value_c = encoder_model.predict(input_seqs)

    # Create batch size empty target sequences
    target_seqs = np.zeros((input_seqs.shape[0], 1))
    # Populate the first character of target sequence with the start token for each one
    target_seqs[:, 0] = target_token_index['START_']

    decoded_sentences = [''] * input_seqs.shape[0]
    stop_conditions = [False] * input_seqs.shape[0]

    while not all(stop_conditions):
        output_tokens, h, c = decoder_model.predict([target_seqs] + [encoder_outputs, states_value_h, states_value_c])

        # Sample tokens for each sequence in the batch
        sampled_token_indices = np.argmax(output_tokens[:, -1, :], axis=1)
        for i in range(input_seqs.shape[0]):
            if not stop_conditions[i]:
                sampled_token = reverse_target_char_index[sampled_token_indices[i]]
                decoded_sentences[i] += ' ' + sampled_token

                # Check for stop condition: either hit max length or find stop token
                if (sampled_token == '_END' or len(decoded_sentences[i]) > max_length_tar):
                    stop_conditions[i] = True

        # Update the target sequence
        target_seqs = np.array(sampled_token_indices)[:, np.newaxis]

        # Update states
        states_value_h, states_value_c = h, c

    return decoded_sentences

# Define a function to convert raw text to padded sequences
def text_to_padded_sequences(texts, tokenizer, maxlen):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=maxlen, padding='post')

# Function to translate new sentences that are not in the dataset
def translate_new_sentences(sentences):
    input_seqs = text_to_padded_sequences(sentences, input_tokenizer, max_length_src)
    decoded_sentences = decode_sequence_batch(input_seqs)
    return decoded_sentences

In [35]:
# Example usage:
new_sentences = ['This is a new sentence.', 'Another example sentence.']
translations = translate_new_sentences(new_sentences)
for translation in translations:
    print(translation)

1/1 [==============================] - 0s 31ms/step
 میں ابھی تک نہیں دے گا۔
 ہم نے بات کی۔ _END


In [36]:
# Define the tokenizer for input sequences
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(lines.eng)

# Function to translate new sentences that are not in the dataset
def translate_new_sentences(sentences):
    input_seqs = text_to_padded_sequences(sentences, input_tokenizer, max_length_src)
    decoded_sentences = decode_sequence_batch(input_seqs)
    return decoded_sentences

# Example usage:
new_sentences = ['hi']
translations = translate_new_sentences(new_sentences)
for translation in translations:
    print(translation)


1/1 [==============================] - 0s 31ms/step
 مدد۔ _END


In [37]:
# Decode sample sequences function using attention
def decode_sequence(input_seq):
    # Encode the input as state vectors and retrieve encoder outputs
    encoder_outputs, h, c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with the start token
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences (assuming a batch of size 1)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [encoder_outputs, h, c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '_END'):
            stop_condition = True

        # Update the target sequence with the sampled token
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [38]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 28ms/step
Input English sentence: the last bus had already gone when i got to the bus stop
Actual Urdu Translation:  میرے بس سٹاپ پہنچنے تک آخری بس نکل چکی تھی۔ 
Predicted Urdu Translation:  میرے بس سٹاپ پہنچنے تک آخری بس نکل چکی تھی۔ 


In [39]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [45]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Urdu Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input English sentence: the snow melted away when spring came
Actual Urdu Translation:  بہار کا موسم آتے ہی برف پگھل گئی۔ 
Predicted Urdu Translation:  وہ خیال میں کوئی نہیں سکتا۔ 
